In [2]:
# DELETE
# Mount your google drive in google colabfrom google.colab import drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# DELETE
# Insert the directory
import sys
sys.path.insert(0,'/content/drive/MyDrive/simbsig/')
# Import worker classes
# from src.TestModule import TestClass
from simbsig.neighbors import KNeighborsClassifier


# KNeighborsClassifier Demo
This demo illustrates how SIMBSIG can be used for Nearest Neighbors classification queries, and how the use compares to scikit-learn. 

In [8]:
# from simbsig.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier as KNeighborsClassifier_sk
import h5py as h5py
import numpy as np
import os

## Set Parameters

In [9]:
n_train_samples = 10
n_classes = 3
n_query_samples = 5
n_dim = 2

## Create Toy Data
### numpy arrays

In [17]:
# numpy arrays
np.random.seed(42)
X_train = np.random.uniform(low=-5, high=5, size=(n_train_samples, n_dim))
X_query = np.random.uniform(low=-5, high=5, size=(n_query_samples, n_dim))
y_train = np.random.randint(low=0, high=n_classes, size=(n_train_samples))

### hdf5 files

In [18]:
# when working on colab, google drive can be used to save and read data
from google.colab import drive
drive.mount('/content/drive')

# depending on the structure of your google drive, you might want to choose a different dataset path
dataset_path = '/content/drive/MyDrive/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# hdf5 files using h5py
train_file = f'train.hdf5'
query_file = f'query.hdf5'

with h5py.File(os.path.join(dataset_path, f"{train_file}"), 'w') as f:
    f.create_dataset("X", data=X_train)
    f.create_dataset("y",data=y_train)

with h5py.File(os.path.join(dataset_path, f"{query_file}"), 'w') as f:
    f.create_dataset("X", data=X_query)

# open the hdf5 file for use
train_data = h5py.File(os.path.join(dataset_path, train_file), 'r')
query_data = h5py.File(os.path.join(dataset_path, query_file))

## Scikit-Learn


In [22]:
nn_sk = KNeighborsClassifier_sk(n_neighbors=2)

nn_sk.fit(X_train, y_train)
nn_sk.predict(X_query)

array([1, 1, 1, 1, 1])

## SIMBSIG
### Using numpy arrays and CPU only
SIMBSIG can be used very similar to scikit-learn. In an existing workflow using scikit-learn, which may be on the verge of exceeding runtime or memory requirements, this allows a seamless transition to SIMBSIG.


In [23]:
nn = KNeighborsClassifier(n_neighbors=2)

nn.fit(X_train, y_train)
nn.predict(X_query)

array([1, 1, 1, 1, 1])

### Using hdf5 files and CPU only
If saving the entire data at once in the computer memory using numpy arrays is not reasonable anymore, the hdf5 file format can help. SIMBSIG can use data in hdf5 files, by setting the `mode` argument to `cpu`

In [24]:
nn_hdf5 = KNeighborsClassifier(n_neighbors=2, mode='hdf5')

nn_hdf5.fit(train_data)
nn_hdf5.predict(query_data)

array([1, 1, 1, 1, 1])

## Using GPU acceleration
If data gets big, the execution time becomes an issue. SIMBSIG features GPU acceleration, by setting the `device` argument to `gpu`. This works with both inputs, numpy arrays and hdf5 files.

In [25]:
nn_hdf5 = KNeighborsClassifier(n_neighbors=2, mode='hdf5', device='gpu')

nn_hdf5.fit(train_data)
nn_hdf5.predict(query_data)

array([1, 1, 1, 1, 1])